In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# IMPORT MODULES

In [ ]:
#PIP INSTALL
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.4/595.4 kB 12.1 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import re
import cv2
import csv
import moviepy.editor
import numpy as np
import matplotlib.pyplot as plt

from time import sleep
from google.colab.patches import cv2_imshow

from ultralytics import YOLO

In [ ]:
from pathlib import Path
from PIL import Image
import torch
from torch import flatten

import torch.nn as nn
from torch.nn import Module, Conv2d, Linear, MaxPool2d, ReLU, LogSoftmax, Dropout

import torchvision.transforms.functional as TF
from torchvision import models
import torch.utils.data as data_utils
from torch.utils.data import Dataset, random_split, DataLoader
from torch.optim import Adam
from torchvision.datasets import KMNIST
#from torchmetrics.functional import accuracy
from torch.nn.utils.rnn import pad_sequence
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import argparse
import imutils
import time
import dlib
from sklearn.preprocessing import normalize
import math
import sys

ROOT = '/code/'
GENERAL_DIR = ROOT + 'FINAL_TEST/'
FRAMES  = GENERAL_DIR + 'FRAMES/'

sys.path.append(ROOT+'mtcnn')#'/content/drive/MyDrive/NAPOLI/mtcnn/') #ROOT+mtcnn
from mtcnn import MTCNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# FUNCTIONS / CLASSES

## **FUNCTIONS EXTERNAL PART**

In [ ]:
# FUNCTION TO REMOVE IMAGES AND LABELS FILE WITHOUT DETECTIONS
def YOLO_rm_empty_file(path, save_dir):
  flag = 0
  for each in os.listdir(path+"labels"):   
    f = open(path+"labels/"+each, "r")
    
    if f.read() == '':
      os.remove(path+"labels/"+each)            #remove file .txt empty
      #os.remove(path+each[:-4]+".png")         #remove images without relevant traffic signs
      #if read file one to one
      os.rmdir(path+"labels/")
      os.rmdir(path)
      flag = 0
      ##########
    else:
      shutil.move(path+"labels/"+each, save_dir)
      #shutil.move(path+each[:-4]+".png", save_dir)
      os.rmdir(path+"labels/")
      os.rmdir(path)
      flag = 1
    f.close()
  return flag

# FUNCTION TO DEVIDE 30 SECOND CLIP IN FRAMES
def getFrames_30clip(vid, output, rate, frameName, flag):   
    vidcap = cv2.VideoCapture(vid)
    clip = moviepy.editor.VideoFileClip(vid)

    seconds = clip.duration
    print('duration: ' + str(seconds))
    
    count = 0
    frame = 0
    
    if not os.path.isdir(output):
        os.mkdir(output)
    
    success = True
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,frame*1000)      
        success, image = vidcap.read()

        ## Stop when last frame is identified
        if frame > seconds or not success:
            break

        print('extracting frame ' + frameName + '-%d.png' % count)
        name = output + '/' + frameName + '-%d.png' % count # save frame as PNG file
        #################
        if int(flag) == 0: crop_img = image
        else: crop_img = image[:,240:1680,:] 
        ################
        cv2.imwrite(name, crop_img)
        frame += rate
        count += 1

#FUNCTION TO EXTRACT THE INFO FROM THE INFO OF TRAFFIC SIGNS' DETECTIONS 
def extraction_from_txt_label(txt_path, width, height):

  f = open(txt_path)

  read = f.read()
  #print(read)
  token = re.split(" |\n", read)
  token = token[:-1]
  #print(token)
  size = len(token)
  #print(size)
  #to normalize values
  info_points = []
  i = 0
  while i < (size):
    i += 1
    center_x = float(token[i]) * width
    i += 1
    center_y = float(token[i]) * height
    i += 1
    box_x = float(token[i]) * width
    i += 1
    box_y = float(token[i]) * height
    i += 1
    info_points.append((center_x, center_y, box_x, box_y))

  return info_points, size//5

# VARIABLES FOR THE VERTICES
def vertex_box(center_x, center_y, box_x, box_y):
  s_s = ( (center_x-(box_x/2)) , (center_y-(box_y/2)) ) #left upper
  i_s = ( (center_x-(box_x/2)) , (center_y+(box_y/2)) ) #left lower
  s_d = ( (center_x+(box_x/2)) , (center_y-(box_y/2)) ) #right upper 
  i_d = ( (center_x+(box_x/2)) , (center_y+(box_y/2)) ) #right lowers

  return s_s, i_s, s_d, i_d

# VALUES OF THE GRID ON THE EXTERNAL IMAGES
def grid(width, height):
  #names of variables with positions of grid, x and y axis
  #z -> zero
  #u -> uno
  #d -> due 
  #t -> tre

  z_x = 0
  z_y = 0

  u_x = width/3       #480
  d_x = (2/3)*width   #960
  t_x = width         #1440

  u_y = height/3      #360
  d_y = (2/3)*height  #720
  t_y = height        #1080

  return z_x, z_y, u_x, d_x, t_x, u_y, d_y, t_y

# FUNCTIONS TO INDICATE THE POSITION AND CENTER OF EACH SING
def detect_center_vertex(XC, YC, s_s, i_s, s_d, i_d, x1_x, x2_x, y1_y, y2_y, str1):
  str_ret = ''
  if x1_x < XC < x2_x  and y1_y < YC < y2_y :  
    #print("CENTER IS IN CELL ", str1)
    str_ret += 'CT'+str1
  #else : str_ret += ' '
  if x1_x < s_s[0] < x2_x  and y1_y < s_s[1] < y2_y :  
    #print("UPPER LEFT END IS IN CELL ", str1)
    str_ret += 'UL'+str1
  #else : str_ret += ' '
  if x1_x < i_s[0] < x2_x  and y1_y < i_s[1] < y2_y :  
    #print("LOWER LEFT END IS IN ", str1) 
    str_ret += 'LL'+str1
  #else : str_ret += ' '
  if x1_x < s_d[0] < x2_x  and y1_y < s_d[1] < y2_y :  
    #print("UPPER RIGHT END IS IN CELL ", str1)
    str_ret += 'UR'+str1
  #else : str_ret += ' '
  if x1_x < i_d[0] < x2_x  and y1_y < i_d[1] < y2_y :  
    #print("LOWER RIGHT END IS IN CELL ", str1)
    str_ret += 'LR'+str1
  #else : str_ret += ' '

  return str_ret 

# FUNCTION FOR EACH CELL, RETURN STRING WITH THE CELL WHERE IS THE SIGNAL
def final_cell(XC, YC, s_s, i_s, s_d, i_d, z_x, u_x, d_x, t_x, z_y, u_y, d_y, t_y):
  
  str_ret = ''  
  str_ret += detect_center_vertex(XC, YC, s_s, i_s, s_d, i_d, z_x, u_x, z_y, u_y, "1")
  str_ret += detect_center_vertex(XC, YC, s_s, i_s, s_d, i_d, u_x, d_x, z_y, u_y, "2")
  str_ret += detect_center_vertex(XC, YC, s_s, i_s, s_d, i_d, d_x, t_x, z_y, u_y, "3")
  str_ret += detect_center_vertex(XC, YC, s_s, i_s, s_d, i_d, z_x, u_x, u_y, d_y, "4")
  str_ret += detect_center_vertex(XC, YC, s_s, i_s, s_d, i_d, u_x, d_x, u_y, d_y, "5")
  str_ret += detect_center_vertex(XC, YC, s_s, i_s, s_d, i_d, d_x, t_x, u_y, d_y, "6")
  str_ret += detect_center_vertex(XC, YC, s_s, i_s, s_d, i_d, z_x, u_x, d_y, t_y, "7")
  str_ret += detect_center_vertex(XC, YC, s_s, i_s, s_d, i_d, u_x, d_x, d_y, t_y, "8")
  str_ret += detect_center_vertex(XC, YC, s_s, i_s, s_d, i_d, d_x, t_x, d_y, t_y, "9")

  return str_ret

# FUNCTION TO DRAW GRID
def draw_rect(image, z_x, u_x, d_x, t_x, z_y, u_y, d_y, t_y, dest_img_path):
  #PARAMEERS: images, lower upper corner, right lower corner, color, thickness

  cv2.rectangle(image, (int(z_x), int(z_y)), (int(u_x), int(u_y)), (0, 255, 0), 5)        #1_1
  cv2.rectangle(image, (int(u_x), int(z_y)), (int(d_x), int(u_y)), (0, 255, 0), 5)        #1_2
  cv2.rectangle(image, (int(d_x), int(z_y)), (int(t_x), int(u_y)), (0, 255, 0), 5)        #1_3
  cv2.rectangle(image, (int(z_x), int(u_y)), (int(u_x), int(d_y)), (0, 255, 0), 5)        #2_1
  cv2.rectangle(image, (int(u_x), int(u_y)), (int(d_x), int(d_y)), (0, 255, 0), 5)        #2_2    
  cv2.rectangle(image, (int(d_x), int(u_y)), (int(t_x), int(d_y)), (0, 255, 0), 5)        #2_3
  cv2.rectangle(image, (int(z_x), int(d_y)), (int(u_x), int(t_y)), (0, 255, 0), 5)        #3_1   
  cv2.rectangle(image, (int(u_x), int(d_y)), (int(d_x), int(t_y)), (0, 255, 0), 5)        #3_2
  cv2.rectangle(image, (int(d_x), int(d_y)), (int(t_x), int(t_y)), (0, 255, 0), 5)        #3_3      
  
  desired_width = 500

  height, width, _ = image.shape
  aspect_ratio = width / height
  new_height = int(desired_width / aspect_ratio)

  resized_img = cv2.resize(image, (desired_width, new_height))

  #image = np.resize(image, 640)

  cv2_imshow(resized_img)
  if dest_img_path != ' ': cv2.imwrite(dest_img_path, image)

# GENERAL FUNCTION, INPUT IMAGE AND TXT, RETURN THE INFO OF THE POSITION
def image_test_cell(image_path, file_txt, dest_img_path):
  
  image = cv2.imread(image_path)
  height, width, c = image.shape
  #print(height, width)

  info_points, n_elem = extraction_from_txt_label(file_txt, width, height)
  ret_celle_string = []
  
  for i in range(n_elem):
    center_x = info_points[i][0]
    center_y = info_points[i][1]
    box_x = info_points[i][2]
    box_y = info_points[i][3]
    #print del cartello detected from YOLO
    j = i+1
    #print(f"{j}° Element detected in photo")
    cv2.rectangle(image, (int(center_x-(box_x/2)), int(center_y-(box_y/2))), (int(center_x+(box_x/2)), int(center_y+(box_y/2))), (255, 0, 0), 5)      
    cv2.circle(image, (int(center_x), int(center_y)), 5, (0,0,255), 5)

    #ESTREMI BOX SIGN
    s_s, i_s, s_d, i_d = vertex_box(center_x, center_y, box_x, box_y)

    z_x, z_y, u_x, d_x, t_x, u_y, d_y, t_y = grid(width, height)

    str_ret = final_cell(center_x, center_y, s_s, i_s, s_d, i_d, z_x, u_x, d_x, t_x, z_y, u_y, d_y, t_y)
    
    #ret_celle_string += str_ret+' '
    ret_celle_string.append(str_ret)

  #print(ret_celle_string)

  draw_rect(image, z_x, u_x, d_x, t_x, z_y, u_y, d_y, t_y, dest_img_path)

  return ret_celle_string

In [ ]:
#list without repetitions of signs' positions 
def uniques_pos(positions):
  list_pos = []
  for elem in positions:
    i = 0
    list_pos_elem = set()
    while i+2 < len(elem):  
      list_pos_elem.add(elem[i+2])
      i = i+3
    list_pos.append(list_pos_elem)
  
  return list_pos 

#number of signs seen
def cell_and_signs(predicted_cell, list_uniques_pos):
  c = 0
  for elem in list_uniques_pos:
    #print(elem)
    if str(predicted_cell) in elem:
      c += 1

  return c

## ****FUNCTIONS INTERNAL PART****

**Head pose estimation**

In [ ]:
class PoseEstimator:
    def __init__(self, weights=None):
      self.detector =  MTCNN()

    def detect_faces(self, image, image_shape_max=640):
      '''
      Parameters
      ----------
      image : uint8 image for face detection.
      image_shape_max : int, optional maximum size (in pixels) of image. The default is None.

      Returns
      -------
      float array bounding boxes and score.
      float array landmarks.

      '''
      image_shape = image.shape[:2]
      
      # perform image resize for faster detection    
      if image_shape_max:
          scale_factor = max([1, max(image_shape) / image_shape_max])
      else:
          scale_factor = 1
          
      if scale_factor > 1:        
          scaled_image = cv2.resize(image, (0, 0), fx = 1/scale_factor, fy = 1/scale_factor)
          
          start = time.time()
          print("[INFO[ performing face detection with MTCNN...")
          boxes, points = self.detector.detect_faces(scaled_image)
          end = time.time()
          print("[INFO] face detection took {:.4f} seconds".format(end - start))

          boxes[:,:4] *= scale_factor
          points *= scale_factor

      else:
          start = time.time()
          print("[INFO[ performing face detection with MTCNN...")
          boxes, points = self.detector.detect_faces(image)
          end = time.time()
          print("[INFO] face detection took {:.4f} seconds".format(end - start))
      
      return boxes, points

    def draw_landmarks(self, image, boxes, points):
      '''
      Parameters
      ----------
      frame : TYPE RGB image
      bb : TYPE - Array of float64, Size = (5,) coordinates of bounding box for the selected face.
      points : TYPE - Array of float32, Size = (10,) coordinates of landmarks for the selected faces.

      Returns
      -------
      None.

      '''
      font = cv2.FONT_HERSHEY_COMPLEX # Text in video
      font_size = 0.6
      blue = (0, 0, 255)
      green = (0,128,0)
      red = (255, 0, 0)

      boxes = boxes.astype(int)
      points = points.astype(int)
      # draw rectangle and landmarks on face
      cv2.rectangle(image, (boxes[0], boxes[1]), (boxes[2], boxes[3]), red, 1)
      cv2.circle(image, (int(points[0]), int(points[5])), 2, blue, 2)# left eye
      cv2.circle(image, (int(points[1]), int(points[6])), 2, blue, 2)# right eye
      cv2.circle(image, (int(points[2]), int(points[7])), 2, blue, 2)# nose
      cv2.circle(image, (int(points[3]), int(points[8])), 2, blue, 2)# mouth - left
      cv2.circle(image, (int(points[4]), int(points[9])), 2, blue, 2)# mouth - right 
      
      cv2_imshow(image)
      
      #w = int(boxes[2])-int(boxes[0]) # width
      #h = int(boxes[3])-int(boxes[1]) # height
      

    def one_face(self, frame, bbs, pointss):
      """
      Parameters
      ----------
      frame : TYPE RGB image (numpy array).
      bbs : TYPE - Array of flaot64, Size = (N, 5) coordinates of bounding boxes for all detected faces.
      pointss : TYPE - Array of flaot32, Size = (N, 10) coordinates of landmarks for all detected faces.
      Returns
      -------
      bb : TYPE - Array of float 64, Size = (5,) coordinates of bounding box for the selected face.
      points : TYPE coordinates of five landmarks for the selected face.
      """
      # select only process only one face (center)
      offsets = [(bbs[:,0]+bbs[:,2])/2-frame.shape[1]/2,
                (bbs[:,1]+bbs[:,3])/2-frame.shape[0]/2]
      offset_dist = np.sum(np.abs(offsets),0)
      index = np.argmin(offset_dist)
      bb = bbs[index]
      points = pointss[:,index]
      return bb, points


    def find_roll(self, points):
      """
      Parameters
      ----------
      points : TYPE - Array of float32, Size = (10,) coordinates of landmarks for the selected faces.
      Returns
      -------
      roll of face.

      """
      return points[6] - points[5]

    def find_yaw(self, points):
      """
      Parameters
      ----------
      points : TYPE - Array of float32, Size = (10,) coordinates of landmarks for the selected faces.
      Returns
      -------
      yaw of face.

      """
      le2n = points[2] - points[0]
      re2n = points[1] - points[2]
      return le2n - re2n

    def find_pitch(self, points):
      """
      Parameters
      ----------
      points : TYPE - Array of float32, Size = (10,) coordinates of landmarks for the selected faces.
      Returns
      -------
      Pitch
      """
      eye_y = (points[5] + points[6]) / 2
      mou_y = (points[8] + points[9]) / 2
      e2n = eye_y - points[7]
      n2m = points[7] - mou_y
      return e2n / n2m


**Process Data for classifier**

In [ ]:
class processData:

  def __init__(self, model):
    self.model = model
    
  def extract_features_from_path(self, path, dest_img_path):
    """
    Parameters
    ----------
    path : path of an image
    dest_img_path : destination path of the image (after cropping)

    Returns
    -------
    face_crop : cropped image (with only the face)
    features : [roll, pitch, yaw, lx, ly, rx, ry] head pose angles + coordinates of the two pupils
    """
    #inizialize the pose estimator
    est = PoseEstimator()

    image = cv2.imread(path)
    image = imutils.resize(image, width=600)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # convert to rgb
    image_rgb = cv2.flip(image_rgb, 1)                  # flip for user friendliness
  
    bounding_boxes, landmarks = est.detect_faces(image_rgb)

    #print(bounding_boxes)

    if len(bounding_boxes) > 0:
      if len(bounding_boxes) >= 2:
        bounding_boxes, landmarks = est.one_face(image_rgb, bounding_boxes, landmarks)       
        roll = est.find_roll(landmarks)
        pitch = est.find_pitch(landmarks)
        yaw = est.find_yaw(landmarks)
        
        lx = landmarks[0]
        ly = landmarks[5]
        rx = landmarks[1]
        ry = landmarks[6]
        
        bounding_boxes = bounding_boxes.astype(int)
        # draw rectangle on face
        x1 = bounding_boxes[0]
        y1 = bounding_boxes[1]
        x2 = bounding_boxes[2]
        y2 = bounding_boxes[3]
        img = cv2.flip(image, 1)
        #cv2.rectangle(img, (bounding_boxes[0], bounding_boxes[1]), (bounding_boxes[2], bounding_boxes[3]), red, 1)
        #cv2_imshow(img)

        w = bounding_boxes[2] - bounding_boxes[0]
        h = bounding_boxes[3] - bounding_boxes[1]
        max_s = max(w, h)
        
        diff_x = (max_s - w)/2
        diff_y = (max_s - h)/2
        
        #to cut the original immages, using face's box
        face_crop = img[int(y1-diff_y):int(y2+diff_y), int(x1-diff_x):int(x2+diff_x)]

        cv2_imshow(face_crop)   
        if dest_img_path != ' ': cv2.imwrite(dest_img_path, face_crop)

      else:
        roll = est.find_roll(landmarks)
        pitch = est.find_pitch(landmarks)
        yaw = est.find_yaw(landmarks)
        lx = landmarks[0]
        ly = landmarks[5]
        rx = landmarks[1]
        ry = landmarks[6]

        bounding_boxes = bounding_boxes.astype(int)
        # draw rectangle on face
        x1 = bounding_boxes[0][0]
        y1 = bounding_boxes[0][1]
        x2 = bounding_boxes[0][2]
        y2 = bounding_boxes[0][3]
        img = cv2.flip(image, 1)
        #cv2.rectangle(img, (bounding_boxes[0][0], bounding_boxes[0][1]), (bounding_boxes[0][2], bounding_boxes[0][3]), red, 1)
        #cv2_imshow(img)

        w = bounding_boxes[0][2] - bounding_boxes[0][0]
        h = bounding_boxes[0][3] - bounding_boxes[0][1]
        max_s = max(w, h)
        
        diff_x = (max_s - w)/2
        diff_y = (max_s - h)/2
        
        #to cut the original immages, using face's box
        face_crop = img[int(y1-diff_y):int(y2+diff_y), int(x1-diff_x):int(x2+diff_x)]

        cv2_imshow(face_crop)
        if dest_img_path != ' ': cv2.imwrite(dest_img_path, face_crop)

    else:
      print("NO FACE FOUND - NEXT EXTERNAL FRAME")
      return [], []

    #FEATURES for NN (head pose, pupils) 

    features = [float(roll), float(pitch), float(yaw), float(lx), float(ly), float(rx), float(ry)]

    return face_crop, features


  def preprocess_for_NN(self, image, features):
    """
    Parameters
    ----------
    image : cropped image of the face
    features : [roll, pitch, yaw, lx, ly, rx, ry] head pose angles + coordinates of the two pupils

    Returns
    -------
    sample : torch tensor that contains the conatenation of image tensor + features  [dim(3, 224, 231)]
    """
    preprocess = transforms.Compose([
                              transforms.ToPILImage(),
                              transforms.Resize(224),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                              ])

    image_tensor = preprocess(image)

    features_tensor = torch.tensor(features).squeeze(-1)
    
    feature = features_tensor.repeat(3, 224, 1)#.squeeze(-1)
    #print(image_tensor.shape, features_tensor.shape)

    sample = torch.cat((image_tensor, feature), 2)  

    #print(sample)
    #print(len(sample))

    return sample


  def gaze_prediction(self, sample, device):
    """
    Cell prediction 

    Parameters
    ----------
    sample : torch tensor that contains the conatenation of image tensor + features  [dim(3, 224, 231)]
    device : torch.device("cuda" if torch.cuda.is_available() else "cpu")

    Returns
    -------
    gaze_cell : number of the predicted cell
    """
    totalTestLoss = 0

    with torch.no_grad():

      x = sample.to(dtype=torch.float32).unsqueeze(0)
      #print(x.shape)

      # send the input to the device
      x = x.to(device)
      # make the predictions and calculate the validation loss
      pred = self.model(x)

      # calculate the number of correct predictions
      gaze_cell = int(pred.argmax(1))+1
      if gaze_cell == 10 : gaze_cell = 0
  
    return gaze_cell

## **Classification Network**

In [ ]:
class HEGClass(Module):
  
  def __init__(self, numChannels, classes):
		# call the parent constructor

    super(HEGClass, self).__init__()  #input(32, 3, 224, 231) 

    self.vgg16 = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
    self.vgg16_featuresextractor = nn.Sequential(*list(self.vgg16.children())[:-1]).to('cuda')

    self.fc6 = Linear(in_features=25088+7, out_features=4096)
    self.relu6_1 = ReLU()
    self.drop6 = Dropout(0.5)

    self.fc7 = Linear(in_features=4096, out_features=1000)
    self.relu7_1 = ReLU()
    self.drop7 = Dropout(0.5)

    self.fc8 = Linear(in_features=1000, out_features=classes)
    self.logSoftmax = LogSoftmax(dim=1)
  
  def forward(self, s): #s.shape = 5x1x224x231

    #print(s.shape)
    x, feature = torch.split(s, [224, 7], dim=3)
    #print(x.shape)
    #print(feature.shape)

    features_images = self.vgg16_featuresextractor(x) 
    
    #print(features_images.shape)
    features_images = flatten(features_images, 1)
    #print(features_images.shape)

    feature = feature[:, 0, 0, :]    #torch.Size([32, 7])
    #print(feature.shape)
    
    x = torch.cat((features_images, feature), 1)
    x = self.fc6(x)
    x = self.relu6_1(x)
    x = self.drop6(x)
    x = self.fc7(x)
    x = self.relu7_1(x)
    x = self.drop7(x)

    x = self.fc8(x)
    output = self.logSoftmax(x)

    return output

## **Functions for csv**

In [ ]:
def init_csv_prediction(path_csv):
  header = []

  header.append("n° of frames")     #file's name - with number of frames
  header.append("n° of signs")      #number of signs present in all the image 
  header.append("signs position")   #uniques number where are center and angles of the images
  header.append("gaze prediction")  #the value of the network
  header.append("n° signs seen")    #if i see those cell how many signs see? number of signs present in the cell that i seeing
  header.append("Attention")        #boolean  

  
  with open(path_csv, 'w') as csv_f:
    writer = csv.writer(csv_f)
    writer.writerow(header) 

In [ ]:
def write_line_csv(file, csv_path, positions, predicted_cell, counter):
    
    set_pos = uniques_pos(positions)

    #number of signs seen when you see one cell
    num_signs = cell_and_signs(predicted_cell, set_pos)

    if num_signs >= 1 or (predicted_cell in [4, 5, 6]):
      att = True
      counter += 1
    elif predicted_cell in [7, 8]: att = 'infotainment'
    else: att = False


    row_csv = [file[9:-4], len(positions), set_pos, predicted_cell, num_signs, att]
    
    print(row_csv)
    
    with open(csv_path, "a") as csv_f:
      #next(csv_f)
      writer = csv.writer(csv_f)
      writer.writerow(row_csv)
      
      csv_f.close()
    return counter


# **EXECUTION PART**

We take in exam an external and internal video of duration of 30 seconds. 

The user can put in *dest_dir_ext* and *dest_dir_ext* any video of any duration, the only important thing is that the 2 videos have to be sinchronized.

If the user want to preprocess his video (make it of 30 seconds), he can find the function in UTILS.ipynb in section 30_SEC SUBCLIPS CUT.

In [ ]:
source_clip_ext = ROOT + 'dataset/DRIVERS-on-DRIVE/EXTERNAL/'  #PATH NAME_CLIP
source_clip_in = ROOT + 'dataset/DRIVERS-on-DRIVE/INTERNAL/'  #PATH NAME_CLIP

dest_dir_ext = FRAMES + 'EXTERNAL/'   #
dest_dir_in = FRAMES + 'INTERNAL/'    #

getFrames_30clip(source_clip_ext, dest_dir_ext, 0.3333, 'external', 1)     
getFrames_30clip(source_clip_in, dest_dir_in, 0.3333, 'internal', 0)     

## Prediction of traffic signs with YOLO v8 and gaze_pose with MTCNN + HEGClass

Initialization of models, weights and saving directory/files paths

In [ ]:
YOLO_weights = ROOT + 'yolo_8_3_classes_best.pt'

HEGClass_weights = ROOT + 'weights_vgg_pretrained_10classes_1002img(5epochs,32batch).pt'

YOLO_model = YOLO(YOLO_weights)  # load a pretrained model 

HEGClass_model = HEGClass(numChannels=3, classes=10).to(device)
HEGClass_model.load_state_dict(torch.load(HEGClass_weights))
HEGClass_model.eval()

In [ ]:
list_file = dest_dir_ext  #directory with frames external clip
save_dir = GENERAL_DIR + "YOLO_Gaze_detection/"

path_csv = GENERAL_DIR + '/ANNOTATIONS/'+ 'evaluation.csv'
#inizializzare writer per csv
init_csv_prediction(path_csv)

In [ ]:
head_gaze_pose = processData(HEGClass_model)

In [ ]:
n_of_att = 0
for file in os.listdir(list_file):
  #print(list_file+file)
  results = YOLO_model.predict(source= list_file+file, save=False, save_txt=True, conf = 0.3)   #decide the value of confidence
  ret = YOLO_rm_empty_file("/content/runs/detect/predict/", save_dir)
  
  if ret == 1:
    image_ext_path = list_file+file  
    print(image_ext_path)
    yolo_txt_path = save_dir+file[:-3]+'txt'

    positions = image_test_cell(image_ext_path, yolo_txt_path, ' ')  #save_dir+file #if third parameter is '' the images with grid is not saved
    #len(positions) 
    print(dest_dir_in)
    image_int_path = dest_dir_in+'internal'+file[8:] 
    print(image_int_path)

    image, features = head_gaze_pose.extract_features_from_path(image_int_path, ' ')  #save_dir+'internal'+file[8:] #if second parameter is '' the images with grid is not saved
    
    if len(image) == 0:
      print('FACE NOT RECOGNIZED')
      continue

    sample = head_gaze_pose.preprocess_for_NN(image, features)
    
    #print(sample)
    
    predicted_cell = head_gaze_pose.gaze_prediction(sample, device)
    
    #print('PREDICTED CELL IS :', predicted_cell)

    n_of_att = write_line_csv(file, path_csv, positions, predicted_cell, n_of_att)

#print('VALORE DI MATCH: ', n_of_att)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#to execute before to evaluate other subclips
for elem in os.listdir(save_dir):
  os.remove(save_dir+elem)

!rm -r /content/runs/detect/predict/labels
!rm -r /content/runs/detect/predict